In [173]:
import pandas as pd
import networkx as nx
import numpy as np
import os
import random
from scipy import sparse
from Dataset.RS_Data_Loader import RS_Data_Loader
from page_rank import pageRank
import importlib
import Support_functions.get_evaluate_data as ged

In [3]:
dataReader = RS_Data_Loader()

URM_train = dataReader.get_URM_train()
URM_validation = dataReader.get_URM_validation()
URM_test = dataReader.get_URM_test()
ICM = dataReader.get_ICM()
UCB = dataReader.get_UCM_matrix_artists()

RS_Data_Loader: loading data...
RS_Data_Loader: loading complete


## Training DataFrame construction for XGBoost

In [224]:
test_best_rec = np.argwhere(URM_train[1,:])[:10][:,1]

In [240]:
# songs of one playlist (fake) recommended
test_best_rec_list

[2018, 3368, 3373, 5024, 5630, 6056, 6209, 6560, 7545, 8050]

In [57]:
current_playlist = 1
del test_best_recc

In [117]:
# The usercontent matrix duplicated for the user of the current playlist
ucb_line = UCB[current_playlist].toarray()[0].reshape((1, -1))

In [119]:
ucb_line.shape

(1, 6668)

In [123]:
np.repeat(ucb_line, 10, axis=0)

(10, 6668)

In [154]:
# ICM for each song
ICM[test_best_rec].toarray().shape

(10, 19412)

In [127]:
res = np.concatenate([ICM[test_best_rec].toarray(), np.repeat(ucb_line, 10, axis=0)], axis=1)

In [153]:
res.shape

(10, 26080)

In [167]:
test_best_rec = test_best_rec.reshape((-1, 1))

In [170]:
test_best_rec.shape

(10, 1)

In [171]:
res = np.concatenate([test_best_rec, res], axis=1)

In [172]:
res.shape

(10, 26081)

In [205]:
user_id = 1
dict_song_pop = ged.tracks_popularity()
user_profile = URM_train.indices[
                                   URM_train.indptr[user_id]:URM_train.indptr[user_id + 1]]

# elements to add for each song

song_pop = np.array([dict_song_pop[user[0]] for user in test_best_rec.tolist()]).reshape((-1,1))
playlist_length = np.array([int(ged.lenght_playlist(user_profile))]*10).reshape((-1,1))
playlist_pop = np.array([int(ged.playlist_popularity(user_profile, pop_dict=dict_song_pop))]*10).reshape((-1,1))

In [206]:
playlist_length.shape

(10, 1)

In [212]:
user_id_array = np.array([1]*10).reshape(-1,1)

In [308]:
user_id_array.shape

(10, 1)

In [309]:
np.concatenate([np.array([1]*10).reshape(-1,1), np.array([1]*10).reshape(-1,1)], axis=0)

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [207]:
res = np.concatenate([res, song_pop, playlist_length, playlist_pop], axis=1)

In [213]:
res = np.concatenate([res, user_id_array], axis=1)

In [214]:
res.shape

(10, 26085)

In [278]:
tracks = dataReader.tracks
tracks_duration = np.array(tracks['duration_sec']).reshape((-1,1))
tracks_duration.shape

(20635, 1)

In [286]:
res_new = np.concatenate([user_id_array, test_best_rec, song_pop, playlist_pop, playlist_length,
                          tracks_duration[test_best_rec_list], ICM[test_best_rec_list].toarray(),
                            np.repeat(ucb_line, 10, axis=0)], axis=1)
res_new.shape

In [301]:
res_sparse = sparse.lil_matrix(res_new, dtype=int)

In [302]:
# that's a coo matrix
res_sparse = sparse.vstack([res_sparse, res_sparse])

In [303]:
res_sparse_csc = res_sparse.tocsc()

In [307]:
df = pd.SparseDataFrame([ pd.SparseSeries(res_sparse_csc[i].toarray().ravel()) 
                                   for i in np.arange(res_sparse_csc.shape[0])])

In [305]:
df

,0,1,2,3,4,5,6,7,8,9,...,26076,26077,26078,26079,26080,26081,26082,26083,26084,26085
0,1,2018,414,211,32,263,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3368,245,211,32,274,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3373,47,211,32,240,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,5024,147,211,32,212,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5630,299,211,32,210,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,6056,21,211,32,188,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,6209,44,211,32,234,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,6560,13,211,32,263,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,7545,569,211,32,178,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,8050,231,211,32,184,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [266]:
df = pd.DataFrame(res_new, dtype=int)

In [250]:
df2 = pd.DataFrame(res_new, dtype=int)

In [267]:
df

,0,1,2,3,4,5,6,7,8,9,...,26075,26076,26077,26078,26079,26080,26081,26082,26083,26084
0,1,2018,414,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3368,245,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,3373,47,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,5024,147,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5630,299,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,6056,21,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,6209,44,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,6560,13,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,7545,569,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,8050,231,211,32,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Page Rank

In [28]:
def compute_PageRank(G, beta=0.85, epsilon=10**-4):
    '''
    Efficient computation of the PageRank values using a sparse adjacency 
    matrix and the iterative power method.
    
    Parameters
    ----------
    G : boolean adjacency matrix. np.bool8
        If the element j,i is True, means that there is a link from i to j.
    beta: 1-teleportation probability.
    epsilon: stop condition. Minimum allowed amount of change in the PageRanks
        between iterations.

    Returns
    -------
    output : tuple
        PageRank array normalized top one.
        Number of iterations.

    '''    
    #Test adjacency matrix is OK
    n,_ = G.shape
    assert(G.shape==(n,n))
    #Constants Speed-UP
    deg_out_beta = G.sum(axis=0).T/beta #vector
    #Initialize
    ranks = np.ones((n,1))/n #vector
    time = 0
    flag = True
    while flag:        
        time +=1
        with np.errstate(divide='ignore'): # Ignore division by 0 on ranks/deg_out_beta
            new_ranks = G.dot((ranks/deg_out_beta)) #vector
        #Leaked PageRank
        new_ranks += (1-new_ranks.sum())/n
        #Stop condition
        if np.linalg.norm(ranks-new_ranks,ord=1)<=epsilon:
            flag = False        
        ranks = new_ranks
    return ranks / ranks.max()

In [69]:
importlib.reload(page_rank.py)

NameError: name 'page_rank' is not defined

In [2]:
dataReader = RS_Data_Loader()

URM_train = dataReader.get_URM_train()
URM_validation = dataReader.get_URM_validation()
URM_test = dataReader.get_URM_test()
#ICM = dataReader.get_ICM()

RS_Data_Loader: loading data...
RS_Data_Loader: loading complete


In [76]:
relation_mat = URM_train.transpose().dot(URM_train).tocsc()

In [77]:
r0 = relation_mat[0].A
r0

array([[19,  0,  0, ...,  0,  0,  0]])

In [78]:
relation_mat

<20635x20635 sparse matrix of type '<class 'numpy.int64'>'
	with 10460699 stored elements in Compressed Sparse Column format>

In [79]:
user_id = 0
songs_in_playlist = URM_train.indices[URM_train.indptr[user_id]:URM_train.indptr[user_id + 1]]

In [80]:
l = range(URM_train.shape[1])
s_all = set(l)

In [81]:
songs_in_playlist

array([ 1220,  8360, 12844, 14301], dtype=int32)

In [82]:
s_0 = s_all - set(songs_in_playlist)

In [83]:
for i in s_0:
    relation_mat.data[relation_mat.indptr[i]:relation_mat.indptr[i+1]].fill(0)
relation_mat.eliminate_zeros()

In [111]:
%%timeit
a = compute_PageRank(relation_mat.transpose())[0]

47.5 ms ± 677 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [86]:
a[1220]

matrix([[0.02241312]])

In [118]:
b = (a / a.max()).A1

In [120]:
b

matrix([[0.0001372],
        [0.0001372],
        [0.0001372],
        ...,
        [0.0001372],
        [0.0001372],
        [0.0001372]])

In [89]:
print(b[1220], b[8360], b[12844])

[[0.11455208]] [[0.95930704]] [[0.20659116]]


In [91]:
URM_train[0] * b

matrix([[2.28045027]])

In [94]:
URM_train[0] * b

array([2.28045027])

In [108]:
URM_train = URM_train.astype(float)

In [109]:
URM_train

<50446x20635 sparse matrix of type '<class 'numpy.float64'>'
	with 949248 stored elements in Compressed Sparse Row format>

In [105]:
b[songs_in_playlist]

array([0.11455208, 0.95930704, 0.20659116, 1.        ])

In [110]:
URM_train[0].data *= b[songs_in_playlist]

In [99]:
URM_train[0].data

array([1, 1, 1, 1])

In [117]:
%%timeit
URM_train[0].data *= b[songs_in_playlist]

58.7 µs ± 4.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
